<a href="https://colab.research.google.com/github/iasminqmoura/ia_2023/blob/main/Trabalho_01_abalone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

nomes_colunas = ['Sexo', 'Comprimento', 'Diametro', 'Altura', 'Peso_Total', 'Peso_Sem_Casca', 'Peso_Visceras', 'Peso_Casca', 'Anéis']
dados_abalone = pd.read_csv('abalone_data.csv', names=nomes_colunas)

codificador_one_hot = OneHotEncoder(sparse_output=False)
sexo_codificado = codificador_one_hot.fit_transform(dados_abalone[['Sexo']])
df_sexo_codificado = pd.DataFrame(sexo_codificado, columns=codificador_one_hot.get_feature_names_out(['Sexo']))

dados_abalone = pd.concat([dados_abalone.drop('Sexo', axis=1), df_sexo_codificado], axis=1)

categorias_tamanho = ['Pequeno', 'Medio', 'Grande']
dados_abalone['Categoria_Tamanho'] = pd.cut(dados_abalone['Comprimento'], bins=[0, 0.4, 0.6, np.inf], labels=categorias_tamanho)

X = dados_abalone.drop('Categoria_Tamanho', axis=1)
y = dados_abalone['Categoria_Tamanho']

normalizador = StandardScaler()
X_normalizado = normalizador.fit_transform(X)

y_codificado = codificador_one_hot.fit_transform(y.values.reshape(-1, 1))

X_treino, X_teste, y_treino, y_teste = train_test_split(X_normalizado, y_codificado, test_size=0.2, random_state=42)

modelo = Sequential([
    Dense(64, activation='relu', input_shape=(X_treino.shape[1],)),
    Dense(32, activation='relu'),
    Dense(y_codificado.shape[1], activation='softmax')
])

modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

historico = modelo.fit(X_treino, y_treino, epochs=50, batch_size=32, validation_split=0.1, verbose=0)

perda, precisao = modelo.evaluate(X_teste, y_teste, verbose=0)

resultados = pd.DataFrame({
    'Conjunto': ['Treino', 'Teste'],
    'Perda': [historico.history['loss'][-1], perda],
    'Precisao (%)': [historico.history['accuracy'][-1] * 100, precisao * 100]
})
print("Exemplo Dataset:")
display(dados_abalone.head().style.set_properties(**{'text-align': 'left'}).set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}]))

print("Treinamento:")
resultados.style.set_properties(**{'text-align': 'left'}).set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}])

Exemplo Dataset:


,Comprimento,Diametro,Altura,Peso_Total,Peso_Sem_Casca,Peso_Visceras,Peso_Casca,Anéis,Sexo_F,Sexo_I,Sexo_M,Categoria_Tamanho
0,0.455000,0.365000,0.095000,0.514000,0.224500,0.101000,0.150000,15,0.000000,0.000000,1.000000,Medio
1,0.350000,0.265000,0.090000,0.225500,0.099500,0.048500,0.070000,7,0.000000,0.000000,1.000000,Pequeno
2,0.530000,0.420000,0.135000,0.677000,0.256500,0.141500,0.210000,9,1.000000,0.000000,0.000000,Medio
3,0.440000,0.365000,0.125000,0.516000,0.215500,0.114000,0.155000,10,0.000000,0.000000,1.000000,Medio
4,0.330000,0.255000,0.080000,0.205000,0.089500,0.039500,0.055000,7,0.000000,1.000000,0.000000,Pequeno


Treinamento:


,Conjunto,Perda,Precisao (%)
0,Treino,0.019976,99.401200
1,Teste,0.026439,98.923445
